In [2]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm, invgamma, gamma
from scipy.linalg import sqrtm
from scipy.optimize import minimize
import pandas as pd
from tqdm import tqdm

# Оценки для опционов колл на отношение цен

Пример (Haug, 2007, с. 203). Вычислите оценки для опционов колл на отношение для
$\sigma_1=0.3$, $\sigma_2=0.4$, $b_1=0.05$,
$b_2=0.03$, $r=0.07$, $S_1=130$, $S_2=100$, $T=0.25, 0.5$, $\rho=\{-0.5, 0, 0.5\}$ и $K=0.1, 0.2, \ldots, 1.0, 2.0, 3.0$, безрисковая ставка 7\%.

In [143]:
sigma1 = 0.3
sigma2 = 0.4
b1 = 0.05
b2 = 0.03
r = 0.07
S1 = 130
S2 = 100
T_range = [0.25, 0.5]
rho_range = [-0.5, 0, 0.5]
K_range = list(np.arange(0.1, 3.1, 0.1))

In [144]:
# оценка стоимости опциона колл на отношение цен двух активов
def quotient_option_price_Zhang(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call):
    theta = 1
    if is_call == False:
        theta = -1
    sigma_hat = np.sqrt(sigma1 ** 2 + sigma2 ** 2 - 2 * rho * sigma1 * sigma2)

    d1 = (np.log(S1 / S2 / K) + (b1 - b2 - 0.5 * (sigma1 ** 2 - sigma2 ** 2)) * T) / \
         (sigma_hat * np.sqrt(T))
    d2 = d1 + sigma_hat * np.sqrt(T)
    F = S1 / S2 * np.exp((b1 - b2 +  sigma2 * (sigma2 - rho * sigma1)) * T)
    option = theta * np.exp(-r * T) * (F * norm.cdf(theta * d2) - K * norm.cdf(theta * d1))

    return option

In [145]:
outperformance_call_df = pd.DataFrame([
    {
        'K': K,
        'rho': rho,
        'time': T,
        'price':quotient_option_price_Zhang(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call = True)
    }
    for K in K_range
    for rho in rho_range
    for T in T_range
])

In [146]:
pd.pivot_table(outperformance_call_df, values='price', index=['K'], columns=['time', 'rho'])

time      0.25                              0.50                    
rho       -0.5       0.0           0.5      -0.5       0.0       0.5
K                                                                   
0.1   1.258176  1.237981  1.218087e+00  1.318772  1.276942  1.236349
0.2   1.159910  1.139716  1.119822e+00  1.222211  1.180382  1.139789
0.3   1.061645  1.041450  1.021556e+00  1.125658  1.083821  1.043228
0.4   0.963381  0.943185  9.232911e-01  1.029201  0.987271  0.946668
0.5   0.865142  0.844921  8.250259e-01  0.933227  0.890826  0.850109
0.6   0.767100  0.746686  7.267608e-01  0.838607  0.794886  0.753578
0.7   0.669880  0.648681  6.285021e-01  0.746640  0.700395  0.657273
0.8   0.574835  0.551674  5.303439e-01  0.658795  0.608852  0.561914
0.9   0.484000  0.457423  4.328959e-01  0.576419  0.522055  0.469130
1.0   0.399660  0.368638  3.382126e-01  0.500532  0.441702  0.381422
1.1   0.323794  0.288321  2.503546e-01  0.431749  0.369086  0.301586
1.2   0.257675  0.218872  1.743351e-01  0.370288  0.304936  0.231923
1.3   0.201735  0.161489  1.138825e-01  0.316041  0.249416  0.173677
1.4   0.155654  0.116063  6.986651e-02  0.268656  0.202224  0.126904
1.5   0.118577  0.081469  4.041277e-02  0.227624  0.162732  0.090696
1.6   0.089341  0.056007  2.215991e-02  0.192349  0.130120  0.063560
1.7   0.066683  0.037811  1.158871e-02  0.162202  0.103489  0.043788
1.8   0.049379  0.025132  5.814785e-03  0.136562  0.081946  0.029725
1.9   0.036324  0.016485  2.815324e-03  0.114840  0.064655  0.019926
2.0   0.026575  0.010692  1.322108e-03  0.096496  0.050866  0.013215
2.1   0.019356  0.006871  6.050017e-04  0.081041  0.039927  0.008687
2.2   0.014049  0.004381  2.708747e-04  0.068047  0.031287  0.005668
2.3   0.010168  0.002776  1.190836e-04  0.057135  0.024487  0.003676
2.4   0.007344  0.001750  5.156523e-05  0.047983  0.019148  0.002372
2.5   0.005296  0.001099  2.205238e-05  0.040310  0.014966  0.001525
2.6   0.003816  0.000688  9.336073e-06  0.033882  0.011695  0.000977
2.7   0.002747  0.000430  3.920709e-06  0.028496  0.009140  0.000625
2.8   0.001978  0.000268  1.636139e-06  0.023983  0.007145  0.000399
2.9   0.001424  0.000167  6.795067e-07  0.020201  0.005588  0.000254
3.0   0.001025  0.000104  2.812284e-07  0.017030  0.004374  0.000162

Пример. Повторите вычисления для опционов пут.

In [147]:
# оценка стоимости опциона колл на отношение цен двух активов
def quotient_option_price_Zhang(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call):
    theta = 1
    if is_call == False:
        theta = -1
    sigma_hat = np.sqrt(sigma1 ** 2 + sigma2 ** 2 - 2 * rho * sigma1 * sigma2)

    d1 = (np.log(S1 / S2 / K) + (b1 - b2 - 0.5 * (sigma1 ** 2 - sigma2 ** 2)) * T) / \
         (sigma_hat * np.sqrt(T))
    d2 = d1 + sigma_hat * np.sqrt(T)
    F = S1 / S2 * np.exp((b1 - b2 +  sigma2 * (sigma2 - rho * sigma1)) * T)
    option = theta * np.exp(-r * T) * (F * norm.cdf(theta * d2) - K * norm.cdf(theta * d1))

    return option

In [148]:
outperformance_put_df = pd.DataFrame([
    {
        'K': K,
        'rho': rho,
        'time': T,
        'price':quotient_option_price_Zhang(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call = False)
    }
    for K in K_range
    for rho in rho_range
    for T in T_range
])

pd.pivot_table(outperformance_put_df, values='price', index=['K'], columns=['time', 'rho'])

time          0.25                                      0.50                \
rho           -0.5           0.0           0.5          -0.5           0.0   
K                                                                            
0.1   3.778084e-20  6.901476e-28  1.250578e-49  5.123091e-12  4.854537e-16   
0.2   2.487462e-12  1.411677e-16  2.177139e-28  7.989515e-08  4.387780e-10   
0.3   9.205951e-09  1.827384e-11  6.814500e-19  7.344569e-06  2.454494e-07   
0.4   1.134986e-06  1.683099e-08  1.869670e-13  1.112457e-04  1.041540e-05   
0.5   2.695529e-05  1.424162e-06  5.708291e-10  6.978822e-04  1.261876e-04   
0.6   2.507371e-04  3.109487e-05  1.365790e-07  2.637833e-03  7.469382e-04   
0.7   1.295519e-03  2.910915e-04  6.678895e-06  7.231218e-03  2.815519e-03   
0.8   4.515951e-03  1.549517e-03  1.137204e-04  1.594718e-02  7.833919e-03   
0.9   1.194655e-02  5.564030e-03  9.309483e-04  3.013163e-02  1.759664e-02   
1.0   2.587163e-02  1.504463e-02  4.512810e-03  5.080542e-02  3.380455e-02   
1.1   4.827076e-02  3.299232e-02  1.492005e-02  7.858253e-02  5.774939e-02   
1.2   8.041701e-02  6.180836e-02  3.716581e-02  1.136823e-01  9.015964e-02   
1.3   1.227419e-01  1.026904e-01  7.497846e-02  1.559958e-01  1.311999e-01   
1.4   1.749267e-01  1.555300e-01  1.292277e-01  2.051716e-01  1.805689e-01   
1.5   2.361140e-01  2.192013e-01  1.980391e-01  2.607001e-01  2.376376e-01   
1.6   3.051435e-01  2.920045e-01  2.780515e-01  3.219852e-01  3.015858e-01   
1.7   3.807513e-01  3.720740e-01  3.657455e-01  3.883986e-01  3.715156e-01   
1.8   4.617120e-01  4.576603e-01  4.582368e-01  4.593193e-01  4.465333e-01   
1.9   5.469222e-01  5.472780e-01  5.535026e-01  5.341583e-01  5.258026e-01   
2.0   6.354385e-01  6.397508e-01  6.502746e-01  6.123745e-01  6.085738e-01   
2.1   7.264853e-01  7.341944e-01  7.478227e-01  6.934807e-01  6.941958e-01   
2.2   8.194428e-01  8.299700e-01  8.457538e-01  7.770463e-01  7.821164e-01   
2.3   9.138275e-01  9.266300e-01  9.438672e-01  8.626954e-01  8.718762e-01   
2.4   1.009269e+00  1.023869e+00  1.042065e+00  9.501033e-01  9.630981e-01   
2.5   1.105486e+00  1.121483e+00  1.140301e+00  1.038992e+00  1.055477e+00   
2.6   1.202271e+00  1.219338e+00  1.238553e+00  1.129124e+00  1.148766e+00   
2.7   1.299467e+00  1.317344e+00  1.336813e+00  1.220298e+00  1.242772e+00   
2.8   1.396963e+00  1.415448e+00  1.435076e+00  1.312346e+00  1.337337e+00   
2.9   1.494674e+00  1.513612e+00  1.533340e+00  1.405125e+00  1.432341e+00   
3.0   1.592541e+00  1.611815e+00  1.631605e+00  1.498514e+00  1.527687e+00   

time                
rho            0.5  
K                   
0.1   3.181683e-27  
0.2   2.895217e-16  
0.3   2.671441e-11  
0.4   2.054317e-08  
0.5   1.562804e-06  
0.6   3.195037e-05  
0.7   2.870075e-04  
0.8   1.489080e-03  
0.9   5.265529e-03  
1.0   1.411781e-02  
1.1   3.084259e-02  
1.2   5.774018e-02  
1.3   9.605428e-02  
1.4   1.458417e-01  
1.5   2.061943e-01  
1.6   2.756193e-01  
1.7   3.524075e-01  
1.8   4.349048e-01  
1.9   5.216662e-01  
2.0   6.115164e-01  
2.1   7.035486e-01  
2.2   7.970902e-01  
2.3   8.916586e-01  
2.4   9.869155e-01  
2.5   1.082629e+00  
2.6   1.178642e+00  
2.7   1.274850e+00  
2.8   1.371184e+00  
2.9   1.467600e+00  
3.0   1.564069e+00

In [149]:
options = pd.merge(outperformance_call_df, outperformance_put_df, on=['K', 'rho', 'time'])
options.columns

Index(['K', 'rho', 'time', 'price_x', 'price_y'], dtype='object')

In [150]:
diff = (options.price_x - options.price_y)
Ks = options.K
Disc = np.exp(- r *options.time)

options['parity'] = (diff + Ks * Disc - S1 / S2 * np.exp((b1 - b2 +  sigma2 * (sigma2 - options.rho * sigma1)) * options.time) * np.exp(-r * options.time))

In [151]:
options.pivot_table(values=['parity'], index='K', columns=['time', 'rho'])
# parity

parity                                                          \
time          0.25                                      0.50                 
rho           -0.5           0.0           0.5          -0.5           0.0   
K                                                                            
0.1  -2.220446e-16  0.000000e+00  0.000000e+00  2.220446e-16  0.000000e+00   
0.2  -2.220446e-16  2.220446e-16  0.000000e+00  0.000000e+00  2.220446e-16   
0.3   0.000000e+00  0.000000e+00  0.000000e+00  2.220446e-16 -2.220446e-16   
0.4   0.000000e+00  2.220446e-16  0.000000e+00  0.000000e+00  0.000000e+00   
0.5   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  2.220446e-16   
0.6   0.000000e+00 -2.220446e-16  0.000000e+00  0.000000e+00  0.000000e+00   
0.7   0.000000e+00  2.220446e-16  0.000000e+00  0.000000e+00 -2.220446e-16   
0.8  -2.220446e-16  2.220446e-16  0.000000e+00  0.000000e+00 -2.220446e-16   
0.9  -2.220446e-16  2.220446e-16  0.000000e+00  0.000000e+00 -2.220446e-16   
1.0   0.000000e+00  2.220446e-16 -2.220446e-16  0.000000e+00  0.000000e+00   
1.1   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
1.2   0.000000e+00  2.220446e-16  0.000000e+00  0.000000e+00  0.000000e+00   
1.3  -2.220446e-16  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
1.4  -2.220446e-16  2.220446e-16 -2.220446e-16  0.000000e+00  0.000000e+00   
1.5   2.220446e-16  2.220446e-16  0.000000e+00 -2.220446e-16 -2.220446e-16   
1.6   0.000000e+00  0.000000e+00  0.000000e+00  2.220446e-16  2.220446e-16   
1.7   0.000000e+00  0.000000e+00  0.000000e+00 -2.220446e-16 -2.220446e-16   
1.8  -2.220446e-16  2.220446e-16  0.000000e+00  0.000000e+00 -2.220446e-16   
1.9   2.220446e-16  2.220446e-16  0.000000e+00  0.000000e+00  0.000000e+00   
2.0   0.000000e+00  2.220446e-16 -2.220446e-16  0.000000e+00 -2.220446e-16   
2.1  -2.220446e-16  0.000000e+00  0.000000e+00  4.440892e-16  2.220446e-16   
2.2   2.220446e-16  2.220446e-16 -2.220446e-16  0.000000e+00 -2.220446e-16   
2.3  -4.440892e-16 -2.220446e-16 -4.440892e-16  0.000000e+00 -2.220446e-16   
2.4  -2.220446e-16  2.220446e-16 -2.220446e-16  0.000000e+00  0.000000e+00   
2.5  -2.220446e-16  0.000000e+00 -2.220446e-16 -2.220446e-16  0.000000e+00   
2.6   2.220446e-16  2.220446e-16 -4.440892e-16  0.000000e+00  0.000000e+00   
2.7   0.000000e+00 -2.220446e-16 -2.220446e-16  0.000000e+00  4.440892e-16   
2.8  -2.220446e-16  0.000000e+00 -4.440892e-16 -2.220446e-16 -2.220446e-16   
2.9   0.000000e+00  0.000000e+00  0.000000e+00  2.220446e-16  2.220446e-16   
3.0   4.440892e-16  2.220446e-16  2.220446e-16 -2.220446e-16 -2.220446e-16   

                    
time                
rho            0.5  
K                   
0.1   0.000000e+00  
0.2   0.000000e+00  
0.3   0.000000e+00  
0.4   0.000000e+00  
0.5   0.000000e+00  
0.6  -2.220446e-16  
0.7   0.000000e+00  
0.8   0.000000e+00  
0.9  -2.220446e-16  
1.0   0.000000e+00  
1.1  -2.220446e-16  
1.2   2.220446e-16  
1.3   0.000000e+00  
1.4  -2.220446e-16  
1.5  -2.220446e-16  
1.6   2.220446e-16  
1.7   0.000000e+00  
1.8  -2.220446e-16  
1.9   2.220446e-16  
2.0   0.000000e+00  
2.1   2.220446e-16  
2.2  -2.220446e-16  
2.3   2.220446e-16  
2.4  -2.220446e-16  
2.5  -2.220446e-16  
2.6   2.220446e-16  
2.7   0.000000e+00  
2.8   2.220446e-16  
2.9   2.220446e-16  
3.0   2.220446e-16

### Задача

# Оценки для опционов на произведение цен

Пример (Haug, 2007, с. 205). Вычислите оценки для опционов колл на произведение цен для $K=15000$, $S_1=100$, $S_2=105$, $b_1=0.02$, $b_2=0.05$, $T=0.5, 1$, $\sigma_1=\{0.2, 0.3, 0.4\}$, $\sigma_2=0.3$, безрисковая ставка $r=0.07$.

In [172]:
K = 15000
b1 = 0.02
b2 = 0.05
r = 0.07
S1 = 100
S2 = 105
T_range = [0.1, 0.5]
rho_range = [-0.5, 0, 0.5]
sigma1_range = [0.2, 0.3, 0.4]
sigma2 = 0.3

In [173]:
#  оценка стоимости опциона колл на произведение цен двух активов
def product_option_price(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call):
    theta = 1
    if is_call == False:
        theta = -1
    sigma_hat = np.sqrt(sigma1 ** 2 + sigma2 ** 2 + 2 * rho * sigma1 * sigma2)

    d1 = (np.log(S1 * S2 / K) + (b1 + b2 - 0.5 * (sigma1 ** 2 + sigma2 ** 2)) * T) / \
         (sigma_hat * np.sqrt(T))
    d2 = d1 + sigma_hat * np.sqrt(T)


    F = S1 * S2 * np.exp((b1 + b2 + rho * sigma1 * sigma2) * T)
    option = theta * np.exp(-r * T) * (F * norm.cdf(theta * d2) - K * norm.cdf(theta * d1))

    return option

In [174]:
prices_call_df = pd.DataFrame([
    {
        'sigma1': sigma1,
        'sigma2': sigma2,
        'rho': rho,
        'time': T,
        'price': product_option_price(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call = True)
    }
    for sigma1 in sigma1_range
    for rho in rho_range
    for T in T_range
])

pd.pivot_table(prices_call_df, values='price', index=['sigma1', 'sigma2'], columns=['time', 'rho'])

time                0.1                              0.5              \
rho                -0.5       0.0        0.5        -0.5         0.0   
sigma1 sigma2                                                          
0.2    0.3     0.002812  0.428850   3.295569   32.613246  154.337957   
0.3    0.3     0.026672  2.402646  13.261771   56.773262  266.159407   
0.4    0.3     0.353503  9.327321  35.490780  118.150427  425.940177   

time                       
rho                   0.5  
sigma1 sigma2              
0.2    0.3     319.714092  
0.3    0.3     531.789411  
0.4    0.3     787.974208

Пример. Повторите вычисления для опционов пут.

In [175]:
#  оценка стоимости опциона пут на произведение цен двух активов
def product_option_price(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call):
    theta = 1
    if is_call == False:
        theta = -1
    sigma_hat = np.sqrt(sigma1 ** 2 + sigma2 ** 2 + 2 * rho * sigma1 * sigma2)

    d1 = (np.log(S1 * S2 / K) + (b1 + b2 - 0.5 * (sigma1 ** 2 + sigma2 ** 2)) * T) / \
         (sigma_hat * np.sqrt(T))
    d2 = d1 + sigma_hat * np.sqrt(T)

    F = S1 * S2 * np.exp((b1 + b2 + rho * sigma1 * sigma2) * T)
    option = theta * np.exp(-r * T) * (F * norm.cdf(theta * d2) - K * norm.cdf(theta * d1))

    return option

In [176]:
prices_put_df = pd.DataFrame([
    {
        'sigma1': sigma1,
        'sigma2': sigma2,
        'rho': rho,
        'time': T,
        'price': product_option_price(S1, S2, b1, b2, sigma1, sigma2, K, rho, T, is_call = False)
    }
    for sigma1 in sigma1_range
    for rho in rho_range
    for T in T_range
])

pd.pivot_table(prices_put_df, values='price', index=['sigma1', 'sigma2'], columns=['time', 'rho'])

time                   0.1                                    0.5  \
rho                   -0.5          0.0          0.5         -0.5   
sigma1 sigma2                                                       
0.2    0.3     4426.822253  4395.795494  4367.114916  4173.019124   
0.3    0.3     4442.537163  4397.769290  4361.271943  4274.466515   
0.4    0.3     4458.531524  4404.693965  4367.668045  4412.553568   

time                                     
rho                    0.0          0.5  
sigma1 sigma2                            
0.2    0.3     4138.419201  4145.108158  
0.3    0.3     4250.240651  4276.942796  
0.4    0.3     4410.021421  4452.282846

### Задача

In [177]:
# выполните проверку через паритет опционов и сделайте выводы

options = pd.merge(prices_call_df, prices_put_df, on=['sigma1', 'sigma2', 'time', 'rho'])
options.columns

Index(['sigma1', 'sigma2', 'rho', 'time', 'price_x', 'price_y'], dtype='object')

In [178]:
diff = (options.price_x - options.price_y)
Disc = np.exp(- r *options.time)

options['parity'] = (diff + K * Disc - S1 * S2 * np.exp((b1 + b2 + options.rho * sigma1 * sigma2) * options.time) * Disc)

In [179]:
options.pivot_table(values=['parity'], index=['sigma1', 'sigma2'], columns=['time', 'rho'])


parity                                           \
time                    0.1                                 0.5        
rho                    -0.5           0.0        0.5       -0.5  0.0   
sigma1 sigma2                                                          
0.2    0.3     1.569105e+01  0.000000e+00 -15.809175  77.287375  0.0   
0.3    0.3    -1.818989e-12  1.818989e-12   0.000000   0.000000  0.0   
0.4    0.3    -1.566753e+01  0.000000e+00  15.832906 -76.709888  0.0   

                             
time                         
rho                     0.5  
sigma1 sigma2                
0.2    0.3    -8.024068e+01  
0.3    0.3     1.818989e-12  
0.4    0.3     8.084475e+01

## Задача

Задача (Zhang, 1998, p. 428-437).
Предположим, что есть две акции со спотовыми ценами $S_1 = S_2 = 100$, волатильностью $\sigma_1 = 18\%$ и $\sigma_2 = 15\%$ и ставками дивидендов $q_1 = 4\%$, $q_2 = 3\%$, коэффициент корреляции доходностей  $\rho=0.75$, безрисковая ставка $r = 5\%$ и цена исполнения опциона $K = 1$.

Оцените цены опционов колл и пут по отношению цены первого актива к цене второго, cрок действия которого истекает через год.

Сравните полученные ответ с книгой и сделайте выводы: call=0.0453, put=0.0557.

In [167]:
# У Чжана другие условия задачи - он подставляет 0.3 и 0.4 в качестве b(=r-q), а не q;
# У Чжана в формуле для d1 вместо (sigma1** - sigma2**2) подставляет sigma_hat**2;
# У Чжана счетная ошибка: пересчитать d1 с его числами - получится другой результат.


def quotient_price_Zhang(tip, S, K, rho, sigma, r, q, T):
    sigma_hat = np.sqrt(sigma[0] ** 2 + sigma[1] ** 2 - 2 * rho * sigma[1] * sigma[0])
    b=[]
    b.append(r-q[0])
    b.append(r-q[1])
    print(b)
    d1 = (np.log(S[0] / S[1] / K) + (b[0] - b[1] - 0.5 * (sigma[0] ** 2 - sigma[1] ** 2)) * T) / \
         (sigma_hat * np.sqrt(T))

    #d1 = (np.log(S[0] / S[1] / K) + (b[0] - b[1] - 0.5 * (sigma_hat**2)) * T) / \
     #    (sigma_hat * np.sqrt(T))
    d2 = d1 + sigma_hat * np.sqrt(T)
    F = S[0] / S[1] * np.exp((b[0] - b[1] +  sigma[1] * (sigma[1] - rho * sigma[0])) * T)

    print(sigma_hat, d1, d2)

    if (tip == 'call'):
        return np.exp(-r * T) * (F * norm.cdf(d2) - K * norm.cdf(d1))
    elif (tip == 'put'):
        return -np.exp(-r * T) * (F * norm.cdf(-d2) - K * norm.cdf(-d1))
    else:
        print('Wrong option type')
        return 0

In [168]:
S=[100.,100.]
K=1
sigma=[0.18,0.15]
q=[0.02,0.01]
rho=0.75
r=0.05
T=1.

print('call=',quotient_price_Zhang('call', S, K, rho, sigma, r, q, T),'; put=',quotient_price_Zhang('put', S, K, rho, sigma, r, q, T))

[0.030000000000000002, 0.04]
0.12000000000000001 -0.12458333333333331 -0.0045833333333333
[0.030000000000000002, 0.04]
0.12000000000000001 -0.12458333333333331 -0.0045833333333333
call= 0.04175792725948984 ; put= 0.049101462345027665


# Мини-проект

Опираясь на книгу ([Hang, 2007](https://drive.google.com/file/d/1yxjRkchVqvl2xkQFyeB2BKNQ1SKJSTtK/view?usp=drive_link)) реализовать расчеты по приведенным формулам и примерам. Привести необходимые определения, оформить формулы с использованием MarkDown. Построить необходимые таблицы и подкрепить полученные результаты иллюстративными графиками. Сделать выводы.

1.   Two-asset correlation options (c. 205).
1.   Exchange-one-asset-for-another options (c. 206).
1.   American exchange-one-asset-for-another options (c. 208).
1.   Exchange options on exchange options (c. 209).
1.   Options on the maximum or the minimun of two risky assets. Call (c. 211).
1.   Options on the maximum or the minimun of two risky assets. Put (c. 211).
1.   Spread-otions approximation (c. 213).
1.   Two-asset barrrier options. Two-asset "out" barriers (c. 215).
1.   Two-asset barrrier options. Two-asset "in" barriers (c. 215).
1.   Partical time two-asset barrrier options. Down-and-in (c. 217).
1.   Partical time two-asset barrrier options. Up-and-in (c. 217).
1.   Two-asset cash-or-nothing options (c. 221)


## Two-asset correlation options (c. 205).

$c = S_2e^{(b_2 - r)T}M(y_2 + \sigma_2\sqrt{T}, y_1 + \rho\sigma_2\sqrt{T}, \rho) - X_2e^{-rT}M(y_2, y_1, \rho)$

$p=X_2e^{-rT}M(-y_2, -y_1, \rho) - S_2e^{(b_2 - r)T}M(-y_2 - \sigma_2\sqrt{T}, -y_1-\rho\sigma_2\sqrt{T}, \rho)$

где:

* $\rho$ - корреляция доходности двух инструментов
* $y_1 = \frac{\ln{\frac{S_1}{X_1}} + (b_1 - 0.5\sigma_1^2)T}{\sigma_1\sqrt{T}}$ 
* $y_2= \frac{\ln{\frac{S_2}{X_2}} + (b_2 - 0.5\sigma_2^2)T}{\sigma_2\sqrt{T}}$

In [185]:
T = 0.5
S1 = 52
S2 = 65
X1 = 50
X2 = 70
sigma1 = .2
sigma2 = .3
r = b1 = b2 = .1
rho = .75
cov = [[1, rho], [rho, 1]]
mean = [0, 0]

In [186]:
from scipy.stats import multivariate_normal

In [192]:
y1 = (np.log(S1 / X1) + (b1 - .5 * sigma1**2) * T) / sigma1 / np.sqrt(T)
y2 = (np.log(S2 / X2) + (b2 - .5 * sigma2**2) * T) / sigma2 / np.sqrt(T)


c = S2 * np.exp((b2 - r)*T) * multivariate_normal.cdf([y2 + sigma2 * np.sqrt(T), 
                                                       y1 + rho * sigma2 * np.sqrt(T)],
                                                      mean=mean, 
                                                      cov=cov) - \
    X2 * np.exp(-r*T) * multivariate_normal.cdf([y2, y1], 
                                                mean=mean, 
                                                cov=cov)

p = X2 * np.exp(-r * T) * multivariate_normal.cdf([-y2, -y1], 
                                                  mean=mean, 
                                                  cov=cov) -\
    S2 * np.exp((b2 - r)*T) * multivariate_normal.cdf([-y2 - sigma2 * np.sqrt(T), 
                                                       -y1 - rho * sigma2 * np.sqrt(T)],
                                                      mean=mean, 
                                                      cov=cov)


print(f'Call option price: {c}')
print(f'Put option price: {p}')

Call option price: 4.707330012666844
Put option price: 3.9092801473640915


## Exchange-one-asset-for-another options (c. 206).

$payoff = max(Q_1S_1 - Q_2S_2, 0)$

$C = Q_1S_1e^{(b_1 - r)T}N(d_1) - Q_2S_2e^{(b_2 -r)T}N(d_2)$

$d_1 = \frac{\ln{\frac{Q_1S_1}{Q_2S_2}} + (b_1 - b_2 + \overline{\sigma}^2 / 2)T}{\overline{\sigma}\sqrt{T}}$

$d_2 = d_1 - \overline{\sigma}\sqrt{T}$

$\overline{\sigma} = \sqrt{\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2}$

In [195]:
S1 = 101
S2 = 104
Q1 = Q2 = 1
T=.5
r = .1
b1 = .02
b2 = .04
sigma1 = .18
sigma2 = .12
rho = .8

In [194]:
from scipy.stats import norm

In [198]:
_sigma = np.sqrt(sigma1**2 + sigma2**2 - 2 * rho * sigma1 * sigma2)
d1 =(np.log(Q1 / Q2 * S1 / S2) + (b1 - b2 + _sigma**2 / 2.)*T) / _sigma / np.sqrt(T)
d2 = d1 - _sigma * np.sqrt(T)

c = Q1 * S1 * np.exp((b1 - r) * T) * norm.cdf(d1) - Q2 * S2 * np.exp((b2 - r)* T) * norm.cdf(d2)
p = Q2 * S2 * np.exp((b2 - r) * T) * norm.cdf(-d2) - Q1 * S1 * np.exp((b1 - r)* T) * norm.cdf(-d1)

print(f'Call option price: {c}')
print(f'Put option price: {p}')

Call option price: 1.5260108471188474
Put option price: 5.412612981779063


## American exchange-one-asset-for-another options (c. 208).

$C = C(Q_1S_1, Q_2S_2, T, r -b_2, b_1 - b_2, \hat{\sigma})$

where $C(S, X, T, r, b, \sigma)$ - American call option

In [199]:
S1 = 22
S2 = 20
Q1 = Q2 = 1
r = .1
b1 = .04
b2 = .06
sigma1 = 0.2
sigma2 = 0.3
rho = .5

In [201]:
def american_option(S, X, T, r, b, sigma, is_call=True):
    d1 = (np.log(S / X) + (b + sigma**2 / 2.)*T) / sigma / np.sqrt(T)
    d2 = d1 - sigma * np.sqrt(T)

    if is_call:
        return S * np.exp((b - r) * T) * norm.cdf(d1) - X * np.exp((b - r)* T) * norm.cdf(d2)
    return X * np.exp((b - r)* T) * norm.cdf(-d2) - S * np.exp((b - r) * T) * norm.cdf(-d1)

_sigma = np.sqrt(sigma1**2 + sigma2**2 - 2 * rho * sigma1 * sigma2)

c = american_option(Q1 * S1, Q2 * S2, T, r-b2, b1 - b2, _sigma)
p = american_option(Q1 * S1, Q2 * S2, T, r-b2, b1 - b2, _sigma, False)

print(f'Call option price: {c}')
print(f'Put oprion price: {p}')

Call option price: 2.189703187789437
Put oprion price: 0.24881212069242142


## Exchange options on exchange options (c. 209).

## Options on the the minimun of two risky assets. Call (c. 211).

$c_{min} = S_1 e^{(b_1 - r)T}M(y_1, -d, \rho_1) + S_2e^{(b_2 - r)T}M(y_2, d-\sigma\sqrt{T}, \rho_2) - Xe^{-rT}M(y_1-\sigma_1\sqrt{T}, y_2 - \sigma_2\sqrt{T}, \rho)$

where:

* $d = \frac{\ln{\frac{S_1}{S_2}} + (b_1 - b_2 + 0.5 \sigma^2)T}{\sigma\sqrt{T}}$
* $y_1 = \frac{\ln{\frac{S1}{X}} + (b_1 + 0.5\sigma_1^2)T}{\sigma_1\sqrt{T}}$
* $y_2 = \frac{\ln{\frac{S2}{X}} + (b_2 + 0.5\sigma_2^2)T}{\sigma_2\sqrt{T}}$
* $\sigma = \sqrt{\sigma_1^2 + \sigma_2^2 - 2 \rho \sigma_1\sigma_2}$
* $\rho_1 = \frac{\sigma_1 - \rho\sigma_2}{\sigma}$
* $\rho_2 = \frac{\sigma_2 - \rho\sigma_1}{\sigma}$

In [202]:
S1 = 100
S2 = 105
X = 98
T = 0.5
r = .05
b1 = -.01
b2 = -.04
sigma1 = .11
sigma2 = .16
rho = .63

In [203]:
sigma = np.sqrt(sigma1**2 + sigma2**2 - 2*rho*sigma1*sigma2)
rho1 = (sigma1 - rho*sigma2) / sigma
rho2 = (sigma2 - rho*sigma1) / sigma

d = (np.log(S1 / S2) + (b1 - b2 + sigma**2 / 2.)*T) / sigma /np.sqrt(T)
y1 = (np.log(S1 / X) + (b1 + sigma1**2 / 2)*T) / sigma1 / np.sqrt(T)
y2 = (np.log(S2 / X) + (b2 + sigma2**2 / 2)*T) / sigma2 / np.sqrt(T)

c = S1 * np.exp((b1 - r)* T) * multivariate_normal.cdf([y1, -d], mean=[0, 0], cov=[[1, rho1], [rho1, 1]]) +\
   S2 * np.exp((b2 - r) * T) * multivariate_normal.cdf([y2, d-sigma* np.sqrt(T)], mean=[0, 0], cov=[[1, rho2], [rho2, 1]]) -\
   X * np.exp(-r*T) * multivariate_normal.cdf([y1 - sigma1*np.sqrt(T), y2 - sigma2 * np.sqrt(T)], mean=[0, 0], cov=[[1, rho], [rho, 1]])

print(f'Call option price: {c}')

Call option price: 25.054682419319136


## Options on the maximum of two risky assets. Call (c. 211).

$c_{max} = S_1e^{(b_1 - r)T}M(y_1, d, \rho_1) + S_2e^{(b_2 - r)T}M(y_2, -d + \sigma*\sqrt{T}, \rho_2) - Xe^{-rT}(1 - M(-y_1 + \sigma_1\sqrt{T}, -y_2 + \sigma_2\sqrt{T}, \rho))$

In [204]:
c = S1 * np.exp((b1 - r)* T) * multivariate_normal.cdf([y1, d], mean=[0, 0], cov=[[1, rho1], [rho1, 1]]) +\
    S2 * np.exp((b2 - r)* T) * multivariate_normal.cdf([y2, -d + sigma*np.sqrt(T)], mean=[0, 0], cov=[[1, rho2],
                                                                                                      [rho2, 1]]) -\
    X * np.exp(-r*T)*(1 - multivariate_normal.cdf([-y1 + sigma1*np.sqrt(T), -y2 + sigma2*np.sqrt(T)],
                                                  mean=[0, 0], cov=[[1, rho], [rho, 1]]))

print(f'Call option price: {c}')

Call option price: 8.070077184968056


## Options on the minimun of two risky assets. Put (c. 211).

$p_{min} = Xe^{-rT} - c_{min}(S_1, S_2, 0, T) + c_{min}(S_1, S_2, X, T)$,

where:  $c_{min}(S_1, S_2, 0, T) = S_1e^{(b_1 - r)T} - S_1e^{(b_1 - r)T}N(d) + S_2e^{(b_2 - r)T}N(d - \sigma\sqrt{T})$

In [205]:
p = X * np.exp(-r*T) - S1 * np.exp((b1 - r)*T) + S1 * np.exp((b1 - r)*T) * norm.cdf(d) -\
    S2 * np.exp((b2 - r)*T) * norm.cdf(d - sigma*np.sqrt(T)) +\
    S1 * np.exp((b1 - r)* T) * multivariate_normal.cdf([y1, -d], mean=[0, 0], cov=[[1, rho1], [rho1, 1]]) +\
    S2 * np.exp((b2 - r) * T) * multivariate_normal.cdf([y2, d-sigma* np.sqrt(T)], mean=[0, 0], 
                                                        cov=[[1, rho2], [rho2, 1]]) -\
    X * np.exp(-r*T) * multivariate_normal.cdf([y1 - sigma1*np.sqrt(T), 
                                                y2 - sigma2 * np.sqrt(T)], 
                                                mean=[0, 0], 
                                                cov=[[1, rho], [rho, 1]])

print(f'Put option price: {p}')

Put option price: 25.643113907654012


## Options on the maximum of two risky assets. Put (c. 211).

$p_{max} = Xe^{-rT} - c_{max}(S_1, S_2, 0, T) + c_{max}(S_1, S_2, X, T)$,

where: $c_{max}(S_1, S_2, 0, T) = S_2e^{(b_2 - r)T} + S_1e^{(b_1 - r)T}N(d) - S_2e^{(b_2 - r)T}N(d - \sigma\sqrt{T})$

In [206]:
p = X * np.exp(-r*T) - S2 * np.exp((b2 - r)*T) - S1 * np.exp((b1 - r)*T)*norm.cdf(d) +\
    S2 * np.exp((b2 - r)*T)*norm.cdf(d - sigma * np.sqrt(T)) +\
    S1 * np.exp((b1 - r)* T) * multivariate_normal.cdf([y1, d], mean=[0, 0], cov=[[1, rho1], [rho1, 1]]) +\
    S2 * np.exp((b2 - r)* T) * multivariate_normal.cdf([y2, -d + sigma*np.sqrt(T)], mean=[0, 0], cov=[[1, rho2],
                                                                                                      [rho2, 1]]) -\
    X * np.exp(-r*T)*(1 - multivariate_normal.cdf([-y1 + sigma1*np.sqrt(T), -y2 + sigma2*np.sqrt(T)],
                                                  mean=[0, 0], cov=[[1, rho], [rho, 1]]))

print(f'Option price: {p}')

Option price: 1.2180995068600424


## Spread-options approximation (c. 213).

$c \approx (Q_2S_2e^{(b2 - r)T} + Xe^{-rT})(SN(d_1) - N(d_2))$

$p \approx (Q_2S_2e^{(b2 - r)T} + Xe^{-rT})(N(-d_2) - SN(-d_1))$

* $d1 = \frac{\ln{S_1} + 0.5 \sigma^2 T}{sigma\sqrt{T}}$
* $d_2 = d_1 - \sigma\sqrt{T}$
* $S = \frac{Q_1S_1e^{(b_1 - r)T}}{Q_2S_2e^{(b_2-r)T} + Xe^{-rT}}$
* $\sigma = \sqrt{\sigma_1^2 + (\sigma_2F)^2 - 2\rho\sigma_1\sigma_2F}$
* $F = \frac{Q_2S_2e^{(b_2 - r)T}}{Q_2S_2e^{(b_2 - r)T} + Xe^{-rT}}$

In [218]:
S1 = 28
S2 = 20
X = 7
T = .25
r =.05
Q1 = Q2 = 1
b1 = b2 = 0.
sigma1 = .29
sigma2 = .36
rho = .42

In [220]:
F = (Q2 * S2 * np.exp((b2 - r)*T)) / (Q2 * S2 * np.exp((b2 - r)*T) + X*np.exp(-r*T))
sigma = np.sqrt(sigma1**2 + sigma2**2 * F**2 - 2*rho*sigma1 * sigma2 * F)
S = (Q1 * S1 * np.exp((b1 - r)*T))/(Q2 * S2 * np.exp((b2 - r)*T) + X*np.exp(-r*T))

d1 = (np.log(S) + .5 * sigma**2 * T) / sigma / np.sqrt(T)
d2 = d1 - sigma * np.sqrt(T)

c = (Q2 * S2 * np.exp((b2 - r)*T) + X * np.exp(-r*T)) * (S * norm.cdf(d1) - norm.cdf(d2))
p = (Q2 * S2 * np.exp((b2 - r)*T) + X * np.exp(-r*T)) * (norm.cdf(-d2) - S * norm.cdf(-d1))

print(f'Call option price: {c}')
print(f'Put option price: {p}')

Call option price: 2.16704845410505
Put option price: 1.1794706536111699


## Two-asset barrrier options. Two-asset "out" barriers (c. 215).

$w = \eta S_1e^{(b_1 - r)T_2}[M(\eta d_1, \phi e_1, -\eta\phi\rho) - exp(\frac{2(\mu_2 + \rho\sigma_1\sigma_2)ln(\frac{H}{S_2})}{\sigma_2^2})M(\eta d_3, \phi e_3, -\eta\phi\rho)] - \eta Xe^{-rT}[M(\eta d_2, \phi e_2, -\eta\phi\rho) - exp(\frac{2\mu_2ln{H}{S_2}}{\sigma_2^2})M(\eta d_4, \phi e_4, -\eta\phi\rho)]$

where:

* $d_1 = \frac{ln(\frac{S_1}{X} + (\mu_1 + \sigma_1^2)T)}{\sigma_1\sqrt{T}}$
* $d_2 = d1 - \sigma_1\sqrt{T}$
* $d_3 = d1 + \frac{2\rho ln(\frac{H}{S_2})}{\sigma_2\sqrt{T}}$
* $d_4 = d2 + \frac{2\rho ln(\frac{H}{S_2})}{\sigma_2\sqrt{T}}$
* $e_1 = \frac{ln(\frac{H}{S_1} - (\mu_2 + \rho\sigma_1\sigma_2)T)}{\sigma_2\sqrt{T}}$
* $e_2 = e_1 + \rho\sigma_1\sqrt{T}$
* $e_3 = e_1 - \frac{2ln(\frac{H}{S_2})}{\sigma_2\sqrt{T}}$
* $e_4 = e_2 - \frac{2ln(\frac{H}{S_2})}{\sigma_2\sqrt{T}}$
* $\mu_1 = b_1 - 0.5\sigma_1^2$
* $\mu_2 = b_2 - 0.5\sigma_2^2$

In [222]:
def two_assets_barriers_option(S1, S2, H, sigma1, sigma2, b1, b2, T, T2, r, rho, is_call, is_out, is_up):
    mu1 = b1 - .5 * sigma1 ** 2
    mu2 = b2 - .5 * sigma2 ** 2
    d1 = (np.log(S1 / X) + (mu1 + sigma1**2)*T) / (sigma1 * np.sqrt(T))
    d2 = d1 - sigma1 * np.sqrt(T)
    d3 = d1 + (2*rho * np.log(H / S2))/(sigma2 * np.sqrt(T))
    d4 = d2 + (2*rho * np.log(H / S2))/(sigma2 * np.sqrt(T))

    e1 = (np.log(H / S2) - (mu2 + rho * sigma1 * sigma2) * T)/(sigma2 * np.sqrt(T))
    e2 = (e1 + rho * sigma1 * np.sqrt(T))
    e3 = e1 - (2 * np.log(H / S2)) / (sigma2 * np.sqrt(T))
    e4 = e2 - (2 * np.log(H / S2)) / (sigma2 * np.sqrt(T))

    eta, phi = None, None


    def call():
        # TODO implement call
        pass

    def put():
        # TODO implement put
        pass

    if is_call:
        if is_out and is_up:
            eta, phi = 1, 1
        elif is_out and not is_up:
            eta, phi = 1, -1
        elif not is_out and is_up:
            pass # call - c_uo
        elif not is_out and not is_up:
            pass # call - c_do
    else:
        if is_out and is_up:
            eta, phi = -1, 1
        elif is_out and not is_up:
            eta, phi = -1, -1
        elif not is_out and is_up:
            pass # put - p_uo
        elif not is_out and not is_up:
            pass # put - p_do


    w = eta * S1 * np.exp((b1 - r)*T2) * (multivariate_normal.cdf([eta * d1, phi * e1], 
                                                                  mean = [0, 0],
                                                                  cov = [[1, -eta * phi * rho],
                                                                         [-eta * phi * rho, 1]]) -\
                                          np.exp((2 * (mu2 + rho*sigma1*sigma2)*np.log(H/S2))/(sigma2**2)) *\
                                          multivariate_normal.cdf([eta*d3, phi*e3],
                                                                  mean=[0, 0],
                                                                  cov=[[1, -eta*rho*phi],
                                                                       [-eta*phi*rho, 1]])) -\
        eta*X*np.exp(-r*T) * (multivariate_normal.cdf([eta*d2, phi*e2],
                                                      mean=[0, 0],
                                                      cov=[[1, -eta*rho*phi],
                                                           [-eta*phi*rho, 1]]) -\
                              np.exp((2*mu2 * np.log(H/S2)) / (sigma2**2)) *\
                              multivariate_normal.cdf([eta*d4, phi*e4],
                                                       mean=[0, 0],
                                                       cov=[[1, -eta*rho*phi],
                                                            [-eta*phi*rho, 1]])
                                                            )
    
    return w

In [225]:
S1 = S2 = 100
T = .5
sigma1 = sigma2 = .2
r = .08
b1 = b2 = .08
X = 90
H = 95
rho = .5

T2 = T

In [227]:
c = two_assets_barriers_option(S1, S2, H, sigma1, sigma2, 
                               b1, b2, T, T2, r, rho, 
                               is_call=True, 
                               is_out=True, 
                               is_up=False)
print(f'Price Down-and-out call: {c}')

c = two_assets_barriers_option(S1, S2, H, sigma1, sigma2, 
                               b1, b2, T, T2, r, rho, 
                               is_call=True, 
                               is_out=True, 
                               is_up=True)
print(f'Price Up-and-out call: {c}')

p = two_assets_barriers_option(S1, S2, H, sigma1, sigma2, 
                               b1, b2, T, T2, r, rho, 
                               is_call=False, 
                               is_out=True, 
                               is_up=False)
print(f'Price Down-and-out put: {p}')

p = two_assets_barriers_option(S1, S2, H, sigma1, sigma2, 
                               b1, b2, T, T2, r, rho, 
                               is_call=False, 
                               is_out=True, 
                               is_up=True)
print(f'Price Up-and-out put: {p}')

Price Down-and-out call: 6.660894064636441
Price Up-and-out call: -1.0699792284166758
Price Down-and-out put: 0.11582647125542844
Price Up-and-out put: -0.7413531967166627


## Two-asset barrrier options. Two-asset "in" barriers (c. 215).

## Partical time two-asset barrrier options. Down-and-in (c. 217).

## Partical time two-asset barrrier options. Up-and-in (c. 217).

# Two-asset cash-or-nothing options (c. 221)